# Out of fold

In [150]:
from data_utils.load import load_data, beautiful_head
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split

In [151]:
def reload_data():
    train = load_data('data/train2016.csv')
    test = load_data('data/test2016.csv')
    y_train = train.Party
    enc_label = LabelEncoder()
    y_train = enc_label.fit_transform(y_train)
    train.drop(['Party', 'USER_ID'], axis = 1, inplace = True)
    test.drop(['USER_ID'], axis = 1, inplace = True)
    
    return train, test, y_train
def binarycoding(train, test, features='all'):
    #import necessary modules
    from sklearn.preprocessing import LabelEncoder
    import numpy as np
    import pandas as pd
    
    #write only categorical names to variable cat_columns
    cat_columns_train = sorted([i for i in train.columns if train[i].dtype == 'O'])
    cat_columns_test = sorted([i for i in test.columns if test[i].dtype == 'O'])
    
    if cat_columns_train != cat_columns_test:
        raise ValueError('Train and test sets must have the same columns')
    
    if features == 'all':
        features = cat_columns_train 
    else:
        if type(features) != list:
            raise ValueError(str(type(features)) + ' type passed in features. Only list are allowed.')
        if len(features) == 0:
            raise ValueError('You have chosen not enough features. The minimum number is one.')
            
        feat_not_in_data = [i for i in features if i not in cat_columns_train]
        
        if len(feat_not_in_data) > 0:
            raise ValueError(','.join(feat_not_in_data) + ' are not in data.')

    #save number of new columns
    labels = []
    l = 0
    
    #do binary coding
    def binary(x):
        max_x = len(bin(max(x))[2:])
        new_x = [list((max_x - len(bin(i)[2:]))*'0' + str(bin(i))[2:]) for i in x]
        return np.array(new_x, dtype = int), max_x
    
    # encode the first column
    enc_label = LabelEncoder()
    new_train, l = binary(enc_label.fit_transform(train[features[0]]))
    labels.append(l)
    new_test,_ = binary(enc_label.transform(test[features[0]]))
    
    # do the others
    for i in features[1:]:
        enc_label = LabelEncoder()
        col, l = binary(enc_label.fit_transform(train[i]))
        new_train = np.column_stack((new_train, col))
        labels.append(l)
        col, l = binary(enc_label.transform(test[i]))
        new_test = np.column_stack((new_test, col))
        
    # create a list of columns to help create a DF from np array 
    new_cols = [features[i] + '_' + str(j) for i in range(0,len(features)) for j in range(labels[i]) ]

    # create new dataframe
    new_train = pd.DataFrame(new_train,columns=new_cols)
    new_test = pd.DataFrame(new_test,columns=new_cols)
    for i in train.columns:
        if i not in features:
            new_train[i] = train[i]
            new_test[i] = test[i]    
    
    return new_train, new_test

In [153]:
train = load_data('data/train2016.csv')
test = load_data('data/test2016.csv')
target = train.Party
enc_label = LabelEncoder()
y = enc_label.fit_transform(y_train)
train.drop(['Party', 'USER_ID'], axis = 1, inplace = True)
test.drop(['USER_ID'], axis = 1, inplace = True)
new_train, new_test = binarycoding(train, test)
X_train, X_test, y_train, y_test = train_test_split(
                                            new_train, y, test_size=0.33, random_state=42, stratify=y)

# XGboost and sklearn for OOF

In [156]:
def oof_xgboost(i, alg, Xtrain, Xtest, Ytrain, mask, test_averaging):
        #function for xgboost
        import xgboost as xgb
        import numpy as np
        import pandas as pd
        
        ansmask = []
        ytrain_i = np.zeros((Xtrain.shape[0]))
        best_iters = []
        Gtest = xgb.DMatrix( Xtest )
        if type(Xtrain) == pd.DataFrame:
            Xtrain = np.array(Xtrain)
        if type(Xtest) == pd.DataFrame:
            Xtest = np.array(Xtest)
        
        for m1, m2 in mask:
            
            Gtrain1 = xgb.DMatrix( Xtrain[m1], label = Ytrain[m1] )
            Gtrain2 = xgb.DMatrix( Xtrain[m2], label = Ytrain[m2] )

            evallist = [(Gtrain1,'train1'), (Gtrain2,'train2')]
            bst = xgb.train( plst, Gtrain1, num_round, evallist, early_stopping_rounds = 10 )
            best_iters += [bst.best_iteration]
            ytrain_i[m2] += bst.predict(Gtrain2)
            ansmask += [bst.predict(Gtest)]

        #Style of test metafeature
        if test_averaging :
            # Average metafeature using K classifiers for each K-1 group of folds
            return ytrain_i, sum(ansmask)/len(mask)
        else :
            # Train single model to get metafeature
            Gtrain = xgb.DMatrix( Xtrain, label = Ytrain )
            bst = xgb.train( plst, Gtrain, int(np.mean(best_iters))+10 )
            ansmask = bst.predict(Gtest)
            return ytrain_i, ansmask

# Class for out-of-fold predictions

In [173]:
class out_of_fold(object):
    
    def __init__(self, n_folds=5, n_times=3, stratified=True, shuffle=True, random_state=1):
        self.n_folds = n_folds
        self.stratified = stratified
        self.random_state = random_state
        self.shuffle = shuffle
        self.n_times = n_times
                
    
    def out_of_fold(self, Xtrain, Xtest, Ytrain, algorithm, test_averaging=True):
        #basic function for creating metaclassifiers
        from sklearn.cross_validation import StratifiedKFold
        
        if algorithm == 'xgboost':
            oof_function = oof_xgboost
        else:
            print algorithm#oof_function = oof_sklearn

        if type(self.n_folds) == int :
            masks = [StratifiedKFold(Ytrain, n_folds=self.n_folds, shuffle=True) for i in range(self.n_times)]
            def ftemp(i) : return oof_function(i, algorithm, Xtrain, Xtest, Ytrain, masks[i], test_averaging=test_averaging)
        else :
            def ftemp(i) : return oof_function(i, algorithm, Xtrain, Xtest, Ytrain, self.n_folds, test_averaging=test_averaging)

        ans = map(ftemp, range(self.n_times))
        return sum([i[0] for i in ans]) / self.n_times, sum([i[1] for i in ans]) / self.n_times
    
    def transform(self, train, test, target, algorithm):
        
        return self.out_of_fold(train, test, target, algorithm)

## Xgboost parameters

In [174]:
param = {}
param['objective'] = 'binary:logistic'
param['max_depth'] = 5
param['eta'] = .1
param['colsample_bytree'] = .75
param['subsample'] = .75
param['nthread'] = 2
plst = list(param.items()) #+ [('eval_metric', 'merror')]
num_round = 1000

# Evaluation

In [175]:
oof = out_of_fold()
train_prediction, metafeature = oof.transform(X_train, X_test, y_train, 'xgboost')

In [179]:
from sklearn.metrics import roc_auc_score as auc
auc(y_train, train_prediction) 

0.66660116727419516

In [180]:
import xgboost as xgb
evallist = [(dtrain,'train'), (dtest,'test')]
dtrain = xgb.DMatrix( X_train, label = y_train )
dtest = xgb.DMatrix( X_test, label = y_test )
bst = xgb.train( plst, dtrain, 100,evallist, early_stopping_rounds = 10 )
test_pred = bst.predict(dtest)

In [181]:
auc(y_test, test_pred)

0.68843697714655105